# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 12 2023 4:16PM,264618,19,ACG-2901000850,ACG North America LLC,Released
1,Jun 12 2023 2:38PM,264604,16,SSF_C1W24.1,Sartorius Stedim Filters Inc.,Released
2,Jun 12 2023 1:59PM,264599,10,SNAP20230608-1E,"Snap Medical Industries, LLC",Released
3,Jun 12 2023 1:25PM,264588,19,ADV80020819,"AdvaGen Pharma, Ltd",Released
4,Jun 12 2023 12:43PM,264582,20,REL1-6411,Reliable 1 Laboratories LLC,Executing
5,Jun 12 2023 12:43PM,264582,20,REL1-6413,Reliable 1 Laboratories LLC,Executing
6,Jun 12 2023 12:43PM,264582,20,REL1-6414,Reliable 1 Laboratories LLC,Executing
7,Jun 12 2023 12:43PM,264582,20,REL1-6415,Reliable 1 Laboratories LLC,Executing
8,Jun 12 2023 12:43PM,264582,20,REL1-6416,Reliable 1 Laboratories LLC,Executing
9,Jun 12 2023 12:43PM,264582,20,REL1-6417,Reliable 1 Laboratories LLC,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
7,264582,Released,1
8,264588,Released,1
9,264599,Released,1
10,264604,Released,1
11,264618,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
264582,NaN,20.0,1.0
264588,NaN,NaN,1.0
264599,NaN,NaN,1.0
264604,NaN,NaN,1.0
264618,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264547,19.0,12.0,0.0
264555,0.0,0.0,1.0
264558,12.0,22.0,6.0
264582,0.0,20.0,1.0
264588,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264547,19,12,0
264555,0,0,1
264558,12,22,6
264582,0,20,1
264588,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264547,19,12,0
1,264555,0,0,1
2,264558,12,22,6
3,264582,0,20,1
4,264588,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264547,19,12,
1,264555,,,1
2,264558,12,22,6
3,264582,,20,1
4,264588,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 12 2023 4:16PM,264618,19,ACG North America LLC
1,Jun 12 2023 2:38PM,264604,16,Sartorius Stedim Filters Inc.
2,Jun 12 2023 1:59PM,264599,10,"Snap Medical Industries, LLC"
3,Jun 12 2023 1:25PM,264588,19,"AdvaGen Pharma, Ltd"
4,Jun 12 2023 12:43PM,264582,20,Reliable 1 Laboratories LLC
25,Jun 12 2023 11:20AM,264558,15,"Brookfield Pharmaceuticals, LLC"
65,Jun 12 2023 10:53AM,264555,19,"NAPP Technologies, LLC"
66,Jun 12 2023 10:10AM,264547,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 12 2023 4:16PM,264618,19,ACG North America LLC,,,1
1,Jun 12 2023 2:38PM,264604,16,Sartorius Stedim Filters Inc.,,,1
2,Jun 12 2023 1:59PM,264599,10,"Snap Medical Industries, LLC",,,1
3,Jun 12 2023 1:25PM,264588,19,"AdvaGen Pharma, Ltd",,,1
4,Jun 12 2023 12:43PM,264582,20,Reliable 1 Laboratories LLC,,20,1
5,Jun 12 2023 11:20AM,264558,15,"Brookfield Pharmaceuticals, LLC",12,22,6
6,Jun 12 2023 10:53AM,264555,19,"NAPP Technologies, LLC",,,1
7,Jun 12 2023 10:10AM,264547,15,"Alliance Pharma, Inc.",19,12,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 12 2023 4:16PM,264618,19,ACG North America LLC,1,,
1,Jun 12 2023 2:38PM,264604,16,Sartorius Stedim Filters Inc.,1,,
2,Jun 12 2023 1:59PM,264599,10,"Snap Medical Industries, LLC",1,,
3,Jun 12 2023 1:25PM,264588,19,"AdvaGen Pharma, Ltd",1,,
4,Jun 12 2023 12:43PM,264582,20,Reliable 1 Laboratories LLC,1,20,
5,Jun 12 2023 11:20AM,264558,15,"Brookfield Pharmaceuticals, LLC",6,22,12
6,Jun 12 2023 10:53AM,264555,19,"NAPP Technologies, LLC",1,,
7,Jun 12 2023 10:10AM,264547,15,"Alliance Pharma, Inc.",,12,19


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 12 2023 4:16PM,264618,19,ACG North America LLC,1,,
1,Jun 12 2023 2:38PM,264604,16,Sartorius Stedim Filters Inc.,1,,
2,Jun 12 2023 1:59PM,264599,10,"Snap Medical Industries, LLC",1,,
3,Jun 12 2023 1:25PM,264588,19,"AdvaGen Pharma, Ltd",1,,
4,Jun 12 2023 12:43PM,264582,20,Reliable 1 Laboratories LLC,1,20,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 12 2023 4:16PM,264618,19,ACG North America LLC,1.0,NaN,NaN
1,Jun 12 2023 2:38PM,264604,16,Sartorius Stedim Filters Inc.,1.0,NaN,NaN
2,Jun 12 2023 1:59PM,264599,10,"Snap Medical Industries, LLC",1.0,NaN,NaN
3,Jun 12 2023 1:25PM,264588,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
4,Jun 12 2023 12:43PM,264582,20,Reliable 1 Laboratories LLC,1.0,20.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 12 2023 4:16PM,264618,19,ACG North America LLC,1.0,0.0,0.0
1,Jun 12 2023 2:38PM,264604,16,Sartorius Stedim Filters Inc.,1.0,0.0,0.0
2,Jun 12 2023 1:59PM,264599,10,"Snap Medical Industries, LLC",1.0,0.0,0.0
3,Jun 12 2023 1:25PM,264588,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
4,Jun 12 2023 12:43PM,264582,20,Reliable 1 Laboratories LLC,1.0,20.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,264599,1.0,0.0,0.0
15,529105,6.0,34.0,31.0
16,264604,1.0,0.0,0.0
19,793761,3.0,0.0,0.0
20,264582,1.0,20.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,264599,1.0,0.0,0.0
1,15,529105,6.0,34.0,31.0
2,16,264604,1.0,0.0,0.0
3,19,793761,3.0,0.0,0.0
4,20,264582,1.0,20.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,1.0,0.0,0.0
1,15,6.0,34.0,31.0
2,16,1.0,0.0,0.0
3,19,3.0,0.0,0.0
4,20,1.0,20.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,1.0
1,15,Released,6.0
2,16,Released,1.0
3,19,Released,3.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20
Status,,,,,
Completed,0.0,31.0,0.0,0.0,0.0
Executing,0.0,34.0,0.0,0.0,20.0
Released,1.0,6.0,1.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20
0,Completed,0.0,31.0,0.0,0.0,0.0
1,Executing,0.0,34.0,0.0,0.0,20.0
2,Released,1.0,6.0,1.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20
0,Completed,0.0,31.0,0.0,0.0,0.0
1,Executing,0.0,34.0,0.0,0.0,20.0
2,Released,1.0,6.0,1.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()